In [14]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


### this cell deals with initialization ########

import matplotlib.pyplot as plt
import numpy as np
from qctrlvisualizer import get_qctrl_style, plot_controls

from qctrl import Qctrl

import json
from io import StringIO


plt.style.use(get_qctrl_style())

# Start a session with the API.
qctrl = Qctrl(email='apteanuj@uchicago.edu', password='nsWvXh4mFCj68x2')


# Define standard deviation of the errors in the experimental results.
sigma = 0.01


segment_count = 100
# duration in nanoseconds
duration = 100 # ns
# no. of shots in each round
shot_count= 50

# Define the number of test points obtained per run.
test_point_count = 25

###############

def costfun(controls):
    experiment_results = qctrl.functions.calculate_qchack_measurements(
        controls=controls,
        shot_count=test_point_count,)
    measurements = experiment_results.measurements     ### actual measurements occur here
    ### let us obtain an array of probabilities for each test_point ###
    cost=np.zeros(test_point_count)
    for j in range(test_point_count):
        #### probabilities are calculated here
        p0 = measurements[j].count(0) / shot_count
        p1 = measurements[j].count(1) / shot_count
        p2 = measurements[j].count(2) / shot_count
        ##cost[j]=cost[j]+np.sqrt(p0**2+(1-p1)**2+p2**2)
        cost[j]=cost[j]+np.sqrt((0.5-p0)**2+(0.5-p1)**2+p2**2)  
    return(cost)



# In[22]:


parameter_set = (
        1.0
        * (np.linspace(-1, 1, test_point_count)[:, None])
        * np.ones((test_point_count, segment_count))
)


####### we are trying to get a test_point_count worth of controls from a parameter_set  ###

def controlsfn(param_set):
    controls = []
    for k in range(test_point_count):
        # Create a random string of complex numbers for each controls.
        values = param_set[k]
        controls.append({"duration": duration, "values": (1.0)*values})
    return(controls)

def costfn(param_set):
    controlset=controlsfn(param_set=param_set)
    return(costfun(controlset))

print(costfn(parameter_set))


# In[23]:



# Define standard deviation of the errors in the experimental results.
sigma = 0.01
# Obtain the cost of initial experimental results.
exp_results = costfn(parameter_set)


length_scale_bound = qctrl.types.closed_loop_optimization_step.BoxConstraint(
    lower_bound=1e-5,
    upper_bound=1e5,
)
bound = qctrl.types.closed_loop_optimization_step.BoxConstraint(
    lower_bound=-1,
    upper_bound=1,
)
initializer = qctrl.types.closed_loop_optimization_step.GaussianProcessInitializer(
    length_scale_bounds=[length_scale_bound] * segment_count,
    bounds=[bound] * segment_count,
    rng_seed=0,
)

# Define state object for the closed-loop optimization.
optimizer = qctrl.types.closed_loop_optimization_step.Optimizer(
    gaussian_process_initializer=initializer,
)


best_cost, best_controls = min(
    zip(exp_results, parameter_set), key=lambda params: params[0]
)
optimization_count = 0

# Run the optimization loop until the cost (infidelity) is sufficiently small.
while optimization_count < 3:
    # Print the current best cost.
    optimization_steps = (
        "optimization step" if optimization_count == 1 else "optimization steps"
    )
    print(
        f"Best infidelity after {optimization_count} BOULDER OPAL {optimization_steps}: {best_cost}"
    )

    # Organize the experiment results into the proper input format.
    results = [
        qctrl.types.closed_loop_optimization_step.CostFunctionResult(
            parameters=list(parameters),
            cost=cost,
            cost_uncertainty=sigma,
        )
        for parameters, cost in zip(parameter_set, exp_results)
    ]

    # Call the automated closed-loop optimizer and obtain the next set of test points.
    optimization_result = qctrl.functions.calculate_closed_loop_optimization_step(
        optimizer=optimizer,
        results=results,
        test_point_count=test_point_count,
    )
    optimization_count += 1

    # Organize the data returned by the automated closed-loop optimizer.
    parameter_set = np.array(
        [test_point.parameters for test_point in optimization_result.test_points]
    )
    optimizer = qctrl.types.closed_loop_optimization_step.Optimizer(
        state=optimization_result.state
    )

    # Obtain experiment results that the automated closed-loop optimizer requested.
    experiment_results = costfn(parameter_set)

    # Record the best results after this round of experiments.
    cost, controls = min(
        zip(experiment_results, parameter_set), key=lambda params: params[0]
    )
    if cost < best_cost:
        best_cost = cost
        best_controls = controls

# Print final best cost.
print(f"Infidelity: {best_cost}")


# Plot controls that correspond to the best cost.
plot_controls(
    figure=plt.figure(),
    controls={
        r"$\Omega(t)$": [
            {"duration": duration*10**(-9)/ len(best_controls), "value": value}
            for value in best_controls
        ]
    },
)




Your task calculate_qchack_measurements has completed in 3s.
[0.48041649 0.35608988 0.35383612 0.42755117 0.44407207 0.46173586
 0.3671512  0.35383612 0.46173586 0.37576588 0.41231056 0.5
 0.5        0.5        0.42755117 0.35383612 0.5        0.3671512
 0.35608988 0.46173586 0.42755117 0.46173586 0.39446166 0.35608988
 0.42755117]


Your task calculate_qchack_measurements has completed in 3s.
Best infidelity after 0 BOULDER OPAL optimization steps: 0.3538361202590827


Your task calculate_closed_loop_optimization_step has started.
Your task calculate_closed_loop_optimization_step has completed in 7s.


Your task calculate_qchack_measurements has completed in 3s.
Best infidelity after 1 BOULDER OPAL optimization step: 0.3538361202590827


Your task calculate_closed_loop_optimization_step has started.
Your task calculate_closed_loop_optimization_step has completed in 12s.


Your task calculate_qchack_measurements has completed in 3s.
Best infidelity after 2 BOULDER OPAL optimization steps: 0.3538361202590827


Your task calculate_closed_loop_optimization_step has started.


KeyboardInterrupt: 

In [ ]:
finalcontrolsarr=[]

for value in best_controls:
     finalcontrolsarr.append({"duration": duration / len(best_controls), "value": value*(1+1j)/np.sqrt(2)})


print(finalcontrolsarr)

def controlsfromparam(best):
    controls = []
    values = best_controls
    controls.append({"duration": duration, "values": (1.0)*values})
    return(controls)

end_test = qctrl.functions.calculate_qchack_measurements(
    controls=controlsfromparam(best_controls),
    shot_count=100,
)

finmeasurements = end_test.measurements

print(finmeasurements)

p0 = finmeasurements[0].count(0) / 100
p1 = finmeasurements[0].count(1) / 100
p2 = finmeasurements[0].count(2) / 100
print(f"P(|0>) = {p0:.2f}, P(|1>) = {p1:.2f}, P(|2>) = {p2:.2f}")


In [ ]:
import jsonpickle.ext.numpy as jsonpickle_numpy
import jsonpickle
import numpy as np

jsonpickle_numpy.register_handlers()
values = best_controls
#not_control = {"duration": duration, "value": (1+1j)*values/np.sqrt(2)}
#json_not_control = jsonpickle.encode(not_control)
#with open("not_gate_pulse.json", "w") as file:
#       file.write(json_not_control)

    
h_control = {"duration": 12.3, "value": (1-1.0j)*values/np.sqrt(2)}
json_h_control = jsonpickle.encode(h_control)
with open("h_gate_pulse.json", "w") as file:
       file.write(json_h_control)